In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import *
from sklearn.metrics         import mean_squared_error
from sklearn.pipeline        import Pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.preprocessing   import OneHotEncoder
from sklearn.preprocessing   import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [ ]:
rgg_rss_data = pd.read_excel('RGG_RSS_final.xlsx')

In [ ]:
rgg_rss_data

,Commodity,Vehicle Type,Inside/Curb,16 gal,20 gal,32 gal,64 gal,96 gal,Meandor,Route,...,Day,#Units,Time,Company,Outlier,Truck,Neighborhood,Address_Street,Code,Has Key
0,Recycle,S-HEIL,C,0,0,1,2,2,NaN,912,...,2,5,118.0,RSS,0,SL,Inner Richmond,5128_geary,NaN,False
1,Recycle,S-HEIL,C,0,0,0,0,1,NaN,912,...,2,1,59.0,RSS,0,SL,Inner Richmond,5620_geary,NaN,False
2,Recycle,S-HEIL,I,0,0,0,0,1,NaN,912,...,2,1,86.0,RSS,0,SL,Inner Richmond,1947_clement,NaN,False
3,Recycle,S-HEIL,C,0,0,1,0,1,NaN,912,...,2,2,41.0,RSS,0,SL,Inner Richmond,1919_clement,NaN,False
4,Recycle,S-HEIL,C,0,0,1,0,0,NaN,912,...,2,1,31.0,RSS,0,SL,Inner Richmond,1909_clement,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,Compost,R-HEIL,I,0,0,0,1,0,NaN,302,...,4,1,98.0,RGG,0,RL,Chinatown,1220_jon,NaN,False
12822,Compost,R-HEIL,I,0,0,0,3,0,NaN,302,...,4,3,100.0,RGG,0,RL,Chinatown,1221_jon,NaN,False
12823,Compost,R-HEIL,C,0,0,0,2,0,NaN,302,...,4,2,66.0,RGG,0,RL,Chinatown,1000_california,NaN,False
12824,Compost,R-HEIL,IC,0,0,0,2,0,NaN,302,...,4,2,118.0,RGG,0,RL,Chinatown,250_eavenworth,NaN,False


In [ ]:
dt = DecisionTreeRegressor(max_depth=8,
             min_samples_leaf=0.13,
            random_state=3)

In [ ]:
col=list(rgg_rss_data.columns)
time_index=col.index("Time")
#print(col.index("Time"))

In [ ]:
data=rgg_rss_data.loc[rgg_rss_data['Outlier']==0]

In [ ]:
X=data
y=data['Time']

In [ ]:
#mode_day=X['Day'].mode()[0]
#X['Day'].replace('na',mode_day,inplace=True)
#X['Day'].replace('nan',mode_day,inplace=True)
#X['Day']=X['Day'].fillna(mode_day)

In [ ]:
X['Day'].unique()

array([2, 5, 3, 1, 4, 'na', nan, 7], dtype=object)

In [ ]:
X=X.drop(columns=['Inside/Curb','Vehicle Type','Meandor','Route','Tipper','Outlier','Address_Street','Code','Time'])
X

,Commodity,16 gal,20 gal,32 gal,64 gal,96 gal,Day,#Units,Company,Truck,Neighborhood,Has Key
0,Recycle,0,0,1,2,2,2,5,RSS,SL,Inner Richmond,False
1,Recycle,0,0,0,0,1,2,1,RSS,SL,Inner Richmond,False
2,Recycle,0,0,0,0,1,2,1,RSS,SL,Inner Richmond,False
3,Recycle,0,0,1,0,1,2,2,RSS,SL,Inner Richmond,False
4,Recycle,0,0,1,0,0,2,1,RSS,SL,Inner Richmond,False
...,...,...,...,...,...,...,...,...,...,...,...,...
12821,Compost,0,0,0,1,0,4,1,RGG,RL,Chinatown,False
12822,Compost,0,0,0,3,0,4,3,RGG,RL,Chinatown,False
12823,Compost,0,0,0,2,0,4,2,RGG,RL,Chinatown,False
12824,Compost,0,0,0,2,0,4,2,RGG,RL,Chinatown,False


In [ ]:
X_dummy=pd.get_dummies(X,columns=['Commodity','Company','Company','Truck','Neighborhood','Has Key','Day'])

In [ ]:
X_dummy

,16 gal,20 gal,32 gal,64 gal,96 gal,#Units,Commodity_Compost,Commodity_Garbage,Commodity_Garbage/Compost,Commodity_Garbage/Recycle,...,Neighborhood_Western Addition,Has Key_False,Has Key_True,Day_1,Day_2,Day_3,Day_4,Day_5,Day_7,Day_na
0,0,0,1,2,2,5,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,0,0,0,0,1,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,0,1,0,1,2,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821,0,0,0,1,0,1,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
12822,0,0,0,3,0,3,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
12823,0,0,0,2,0,2,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
12824,0,0,0,2,0,2,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [ ]:
y

0        118.0
1         59.0
2         86.0
3         41.0
4         31.0
         ...  
12821     98.0
12822    100.0
12823     66.0
12824    118.0
12825    180.0
Name: Time, Length: 12769, dtype: float64

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_dummy, y, test_size = 0.2, random_state=16)

In [ ]:
dt = DecisionTreeRegressor(max_depth=8,
             min_samples_leaf=0.13,
            random_state=3)

In [ ]:
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
y_pred = dt.predict(X_validation)

In [ ]:
mse_dt = MSE(y_validation, y_pred)


In [ ]:
rmse_dt = mse_dt**(1/2)

In [ ]:
print("Test set RMSE of dt: {:.2f}".format(rmse_dt))

Test set RMSE of dt: 56.15


### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=200)
rfr.fit(X_train, y_train)
print(rfr.score(X_train, y_train))
print(rfr.score(X_validation, y_validation))


0.7623345009104382
0.5589588768228396


In [ ]:
rfr.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
hyperparameter={'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 60,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 97,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
rfr_tuned = RandomForestRegressor(**hyperparameter)
rfr.fit(X_train, y_train)
print(rfr.score(X_train, y_train))
print(rfr.score(X_validation, y_validation))

0.7622743814296966
0.5609163644589128


In [ ]:
rfr = RandomForestRegressor(n_estimators=200)
rfr.fit(X_train, y_train)
print(rfr.score(X_train, y_train))
print(rfr.score(X_validation, y_validation))

In [ ]:
a=[]
for x,y in zip(X_train.columns,rfr.feature_importances_):
    a.append((x,y))

In [ ]:
a.sort(key=lambda x:x[1],reverse=True)
a

[('96 gal', 0.4096645808023728),
 ('#Units', 0.19991315715912256),
 ('64 gal', 0.042709197684657),
 ('Neighborhood_Nob Hill', 0.02941212375495386),
 ('32 gal', 0.022212005056344207),
 ('Day_1', 0.018146247539570812),
 ('Neighborhood_Financial District', 0.01698957676219288),
 ('Neighborhood_Pacific Heights', 0.015540764958431687),
 ('Has Key_True', 0.01541004285533351),
 ('Commodity_Recycle', 0.015057466434372167),
 ('Has Key_False', 0.014687521512077924),
 ('Commodity_Garbage', 0.013876096322805344),
 ('Truck_RL', 0.013765121083179156),
 ('Neighborhood_Russian Hill', 0.013617875712022965),
 ('Day_3', 0.013596913704812162),
 ('Neighborhood_Inner Richmond', 0.013210766709418804),
 ('Day_4', 0.012891238400451947),
 ('Commodity_Garbage/Compost', 0.011177282176837172),
 ('Truck_SL', 0.010788760477722125),
 ('Commodity_Garbage/Recycle', 0.010761486451962362),
 ('Day_2', 0.01069726027851928),
 ('Day_5', 0.010206879363004896),
 ('Neighborhood_North Beach', 0.009248293863825821),
 ('16 gal', 0

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 60, stop = 400, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}



In [ ]:
rf = RandomForestRegressor()
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [60, 97, 135, 173, 211,
                                                         248, 286, 324, 362,
                                                         400]},
                   random_state=42, verbose=2)

In [ ]:
rf_random.best_estimator_.get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 60,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 97,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
evaluate()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=938c6ad9-491d-4307-bf8a-c751a244ce4f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>